# Imports and config

In [ ]:
import os
import dspy
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify API key is loaded
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("API key loaded successfully")
else:
    print("API key not found in environment variables")


# Configure the LM (Language Model)

In [ ]:
# Configure the LM (Language Model) with structured output support
lm = dspy.LM("openai/gpt-4o-mini", model_type="chat")
dspy.settings.configure(lm=lm)

## Use DSPy built-in Module to Build a Sentiment Classifier

In [ ]:
class SentimentClassifier(dspy.Signature):
    """Classify the sentiment of a text."""

    text: str = dspy.InputField(desc="input text to classify sentiment")
    sentiment: int = dspy.OutputField(
        desc="sentiment, the higher the more positive", ge=0, le=10
    )

# For my own notes: ge and le are pydantic constraints restricting the range of the sentiment output between 0-10

In [ ]:
str_signature = dspy.make_signature("text -> sentiment")

## Create a Module to Interact with the LM

In [ ]:
# Test the sentiment classifier
try:
    predict = dspy.Predict(SentimentClassifier)
    output = predict(text="I am feeling pretty happy about this!")
    print("Prediction successful!")
    print(f"Text: 'I am feeling pretty happy about this!'")
    print(f"Sentiment: {output.sentiment}")
except Exception as e:
    print(f"Error occurred: {e}")
    print("Please check your API key and internet connection.")

In [ ]:
# Test with different sentiment examples
test_texts = [
    "I am feeling pretty happy about this!",
    "This is terrible and I hate it.",
    "I feel neutral about this situation.",
    "I'm absolutely thrilled with the results!",
    "This makes me so angry and frustrated."
]

print("Testing sentiment classification with multiple examples:")

for text in test_texts:
    try:
        output = predict(text=text)
        print(f"Text: '{text}'")
        print(f"Sentiment: {output.sentiment}/10")
        print("." * 40)
    except Exception as e:
        print(f"Error processing '{text}': {e}")
        print("." * 40)


In [ ]:
print(f"The sentiment is: {output.sentiment}")
print(f"The sentiment is {output['sentiment']}")

In [ ]:
dspy.configure(lm=dspy.LM("openai/gpt-4o"))
print(predict(text="I am feeling pretty happy!"))

In [ ]:
dspy.configure(lm=dspy.LM("openai/gpt-4o-mini"))

Where is my prompt?
Check dspy.inspect_history(n=1) where n is how many entries you want to pull from the memory.

In [ ]:
dspy.inspect_history(n=1)

# Chain of Thought Built-in module.

In [ ]:
cot = dspy.ChainofThought(SentimentClassifier)

output = cot(text="I am feeling pretty happy!")
print(output)

In [ ]:
dspy.inspect_history(n=1)

## Using a different Adapter

In [ ]:
dspy.configure(adapter=dspy.JSONAdapter())

In [ ]:
print(cot(text="I am feeling pretty happy!"))
dspy.inspect_history(n=1)

## Building a Program with Custom Module

In [ ]:
class QuestionGenerator(dspy.Signature):
    """Genereate a yes or no question in order to guess the celebrity name"""
    past_questions: list[str] = dspy.InputField(desc="past questions ")
    past_answers: list[bool] = dspy.InputField(desc="past answers")
    new_question: str = dspy.OutputField(desc="new question that can")
    guess_made: bool = dspy.OutputField(desc="If the new_question is")


class Reflection(dspy.Signature):
    """Provide reflection on the guessing process"""
    correct_celebrity_name: str = dspy.InputField(desc="the celebrity")
    final_guessor_question: str = dspy.InputField(desc="the final guess")
    past_questions: list[str] = dspy.InputField(desc="past questions ")
    past_answers: list[bool] = dspy.InputField(desc="past answers")

    reflection: str = dspy.OytputField(
        desc="reflection on the guessing process, including what was"
    )

def ask(prompt, valid_responses=("y", "n")):
    while True:
        response = input(f"{prompt} ({'/'.join(valid_responses)}): ")
        if response in valid_responses:
            return response
        print(f"Please enter one of: {', '.join(valid_responses)}")


class CelebrityGuess(dspy.Module):
    def __init__(self, max_tries=10):
        super().__init__()

        self.question_generator = dspy.ChainOfThought(QuestionGenerator)
        self.reflection = dspy.ChainofThought(Reflection)

        self.max_tries = 20

    def forward(self):
        celebrity_name = input("Please think of a celebrity name, once ")
        past_questions = []
        past_answers = []

        correct_guess = False

        for i in range(self.max_tries):
            question = self.question_generator(
                past_questions=past_questions,
                past_answers=past_answers,
            )
            answer = ask(f"{question.new_question}").lower() == "y"
            past_questions.append(question.new_question)
            past_answers.append(answer)

            if question.guess_made and answer:
                correct_guess = True
                break

        if correct_guess:
            print("Yay! I got it right!")
        else:
            print("Oops, I couldn't gguess it right.")

        reflection = self.reflection(
            correct_celebrity_name=celebrity_name,
            final_guessor_question=question.new_question,
            past_questions=past_questions,
            past_answers=past_answers,
        )
        print(reflection.reflection)

In [ ]:
celebrity_guess = CelebrityGuess()
celebrity_guess